In [38]:
import importlib

import plotly.graph_objects as go
from plotly.subplots import make_subplots
from statsmodels.tsa.stattools import adfuller

import apollo.api.yahoo_api_connector as yac
import apollo.utils.time_series_transformer as tst

importlib.reload(yac)
importlib.reload(tst);

In [39]:
from apollo.settings import END_DATE, START_DATE, TICKER


In [40]:
api_connector = yac.YahooApiConnector(
    ticker=str(TICKER),
    start_date=str(START_DATE),
    end_date=str(END_DATE),
)

dataframe = api_connector.request_or_read_prices()

dataframe

,ticker,open,high,low,close,adj close,volume
date,,,,,,,
2007-01-03,SPY,142.250000,142.860001,140.570007,141.369995,101.536606,94807600
2007-01-04,SPY,141.229996,142.050003,140.610001,141.669998,101.752022,69620600
2007-01-05,SPY,141.330002,141.399994,140.380005,140.539993,100.940468,76645300
2007-01-08,SPY,140.820007,141.410004,140.250000,141.190002,101.407318,71655000
2007-01-09,SPY,141.309998,141.600006,140.399994,141.070007,101.321114,75680100
...,...,...,...,...,...,...,...
2012-12-24,SPY,142.479996,142.559998,142.190002,142.350006,116.096169,53874600
2012-12-26,SPY,142.639999,142.710007,141.350006,141.750000,115.606819,106947700
2012-12-27,SPY,141.789993,142.080002,139.919998,141.559998,115.451851,167920600


In [41]:
dataframe["close"] = tst.TimeSeriesTransformer.bring_to_stationary(dataframe["close"])

dataframe.dropna(inplace=True)

In [42]:
adf_test = adfuller(dataframe["close"], autolag="AIC", regression="ct")

print("Null Hypothesis: The series has a unit root (non-stationary)")
print("ADF-Statistic:", adf_test[0])
print("P-Value:", adf_test[1])
print("Number of lags:", adf_test[2])
print("Number of observations:", adf_test[3])
print("Note: If P-Value is smaller than 0.05, we reject the null hypothesis and the series is stationary")

Null Hypothesis: The series has a unit root (non-stationary)
ADF-Statistic: -30.729728934396643
P-Value: 0.0
Number of lags: 1
Number of observations: 1507
Note: If P-Value is smaller than 0.05, we reject the null hypothesis and the series is stationary


In [43]:
fig = make_subplots(rows=1, cols=1, specs=[[{}]], vertical_spacing=0)

# Plot closing prices
fig.add_trace(
    go.Scatter(
        x=dataframe.index,
        y=dataframe["close"],
        line={"color":"blue", "width":1},
        name="Close",
    ),
    row=1,
    col=1,
)

fig.update_yaxes(visible=False, secondary_y=True)